In [1]:
CONFIG = {
    'data_folder' : 'C:/Users/Николай/PycharmProjects/VKRecSys/data/',
    'models_folder' : 'C:/Users/Николай/PycharmProjects/VKRecSys/B.Processing/Модели/',
    
    'train_path' : 'train_interactions.parquet',
    'items_meta_path' : 'items_meta.parquet',
    'users_meta_path' : 'users_meta.parquet',
    'model_path' : '4.2.pth',
    
    'user_emb_size' : 256, # 183404
    'item_emb_size' : 256, # 337727
    'source_emb_size' : 256, # 19613
    'torch_precision' : 40, # number of decimal places for printing numbers
    
    'DEVICE' : 'cuda',
    'SEED' : 42,
    'BATCH_SIZE' : 16384,
    'LR' : 0.0001,
    'EPOCHS' : 3,
    'output_dim' : 3
    
}

In [2]:
# Import libs
import numpy as np
from tqdm import tqdm
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import OneCycleLR

In [3]:
# Device and seed
device = torch.device(CONFIG['DEVICE'] if torch.cuda.is_available() else "cpu")
torch.set_printoptions(precision=CONFIG['torch_precision']) 

torch.manual_seed(CONFIG['SEED'])  
torch.cuda.manual_seed_all(CONFIG['SEED'])  
np.random.seed(CONFIG['SEED'])  

In [4]:
# Loading data
train = pd.read_parquet(f"{CONFIG['data_folder']}{CONFIG['train_path']}", engine='pyarrow')
train['like'] = train['like'] + train['dislike'].replace({1: -1})
train.drop(columns=['dislike'], inplace=True)
train['like'] = train['like'].astype('int8')
train.rename(columns={'like' : 'target'}, inplace=True)
train['target'] = train['target'].replace({-1:0, 0:1, 1:2})

items_meta = pd.read_parquet(f"{CONFIG['data_folder']}{CONFIG['items_meta_path']}", engine='pyarrow')
items_meta['item_id'] = items_meta['item_id'].astype('category')
items_meta['source_id'] = items_meta['source_id'].astype('category')
items_meta.set_index('item_id', inplace=True)

users_meta = pd.read_parquet(f"{CONFIG['data_folder']}{CONFIG['users_meta_path']}", engine='pyarrow')
users_meta['user_id'] = users_meta['user_id'].astype('category')
users_meta['gender'] = users_meta['gender'].astype('category')
users_meta['gender'] = users_meta['gender'].replace({1:0, 2:1})
users_meta.set_index('user_id', inplace=True)

In [5]:
# Normalization of numeric features
users_meta['age'] = (users_meta['age'] - users_meta['age'].min()) / (users_meta['age'].max() - users_meta['age'].min())
items_meta['duration'] = (items_meta['duration'] - items_meta['duration'].min()) / (items_meta['duration'].max() - items_meta['duration'].min())

In [6]:
# Model definition
class MLPModel(nn.Module):
    def __init__(self, 
                 input_dim, 
                 num_users=users_meta.index.nunique(), 
                 num_items=items_meta.index.nunique(), 
                 num_sources=items_meta['source_id'].nunique(),
                 output_dim=CONFIG['output_dim'],
                 dropout_rate=0.2):  # Добавлен параметр dropout_rate
        
        super(MLPModel, self).__init__()
        self.user_embedding = nn.Embedding(num_users, CONFIG['user_emb_size'])
        self.item_embedding = nn.Embedding(num_items, CONFIG['item_emb_size'])
        self.source_embedding = nn.Embedding(num_sources, CONFIG['source_emb_size'])

        self.fc1 = nn.Linear(input_dim, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.fc4 = nn.Linear(512, 256)
        self.fc5 = nn.Linear(256, 256)
        self.fc6 = nn.Linear(256, 128)
        self.fc7 = nn.Linear(128, 128)
        self.fc8 = nn.Linear(128, 64)
        self.fc9 = nn.Linear(64, output_dim)
        
        self.softplus = nn.Softplus()

    def forward(self, user_ids, item_ids, source_ids, age, duration, gender, embeddings):
        user_emb = self.user_embedding(user_ids)
        item_emb = self.item_embedding(item_ids)
        source_emb = self.source_embedding(source_ids)

        x = torch.cat((user_emb, item_emb, source_emb, age, duration, gender, embeddings), dim=1)
        
        x = self.softplus(self.fc1(x))
        
        x = self.softplus(self.fc2(x))
        
        x = self.softplus(self.fc3(x))
        
        x = self.softplus(self.fc4(x))
        
        x = self.softplus(self.fc5(x))
        
        x = self.softplus(self.fc6(x))
        
        x = self.softplus(self.fc7(x))
        
        x = self.softplus(self.fc8(x))
        
        x = self.fc9(x)
        
        return x

In [7]:
# Input dimension
input_dim = 1 + 1 + 1 + CONFIG['user_emb_size'] + CONFIG['item_emb_size'] + CONFIG['source_emb_size'] + 32

In [8]:
# Model creation
model = MLPModel(input_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=CONFIG['LR'])

In [9]:
# Train
num_samples = len(train)
num_batches = (num_samples + CONFIG['BATCH_SIZE'] - 1) // CONFIG['BATCH_SIZE']

lr_scheduler = OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=num_batches, epochs=CONFIG['EPOCHS'])

for epoch in range(CONFIG['EPOCHS']):
    running_loss = 0.0
    with tqdm(range(num_batches), desc=f"Epoch {epoch+1}/{CONFIG['EPOCHS']}", unit="batch") as t:
        for batch_idx in t:
            start_idx = batch_idx * CONFIG['BATCH_SIZE']
            end_idx = min(start_idx + CONFIG['BATCH_SIZE'], num_samples)
            batch = train.iloc[start_idx:end_idx]

            batch_user_ids = torch.tensor(batch['user_id'].values, dtype=torch.long, device=device)
            batch_item_ids = torch.tensor(batch['item_id'].values, dtype=torch.long, device=device)
            batch_source_ids = torch.tensor(items_meta.loc[batch['item_id'].values, 'source_id'].cat.codes.values, dtype=torch.long, device=device)
            batch_age = torch.tensor(users_meta.loc[batch['user_id'].values, 'age'].values, dtype=torch.float32, device=device).unsqueeze(1)    
            batch_duration = torch.tensor(items_meta.loc[batch['item_id'].values, 'duration'].values, dtype=torch.float32, device=device).unsqueeze(1)
            batch_gender = torch.tensor(users_meta.loc[batch['user_id'].values, 'gender'].cat.codes.values, dtype=torch.float32, device=device).unsqueeze(1)

            item_indices = batch_item_ids.cpu().numpy()
            embeddings = torch.tensor(np.stack(items_meta.loc[item_indices, 'embeddings'].values), device=device, dtype=torch.float32)

            targets = torch.tensor(batch['target'].values, dtype=torch.long, device=device)

            optimizer.zero_grad()
            outputs = model(batch_user_ids, batch_item_ids, batch_source_ids, batch_age, batch_duration, batch_gender, embeddings)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            lr_scheduler.step()
            
            running_loss += loss.item()
            t.set_postfix(mean_loss=f"{running_loss / (batch_idx + 1):.6f}")

    print(f"Epoch [{epoch+1}/{CONFIG['EPOCHS']}], Loss: {running_loss / num_batches:.4f}")

Epoch 1/3: 100%|██████████| 8891/8891 [44:49<00:00,  3.31batch/s, mean_loss=0.136550]


Epoch [1/3], Loss: 0.1366


Epoch 2/3: 100%|██████████| 8891/8891 [44:39<00:00,  3.32batch/s, mean_loss=0.124516]


Epoch [2/3], Loss: 0.1245


Epoch 3/3: 100%|██████████| 8891/8891 [43:50<00:00,  3.38batch/s, mean_loss=0.120124]

Epoch [3/3], Loss: 0.1201


In [10]:
# Save model_state  
torch.save({"model_state_dict": model.state_dict()}, f"{CONFIG['models_folder']}{CONFIG['model_path']}")
print(f"Модель сохранена в f'{CONFIG['models_folder']}{CONFIG['model_path']}'")

Модель сохранена в f'C:/Users/Николай/PycharmProjects/VKRecSys/B.Processing/Модели/4.2.pth'
